<h4>ДЗ</h4>
<p>1. Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.</p>

In [44]:
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [45]:
from sklearn.datasets import load_iris

In [46]:
load_iris()

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [47]:
iris_data=load_iris()['data']

In [48]:
iris_target=load_iris()['target']

In [49]:
X_train,X_validation,y_train,y_validation=train_test_split(iris_data,iris_target,test_size=0.25,random_state=42)

In [50]:
#Реализуем дерево решений

In [51]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

In [52]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):

        classes = {} 
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
           
        prediction = max(classes, key=classes.get)
        return prediction        

In [53]:


def gini(labels):
    
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [54]:


def gain(left_labels, right_labels, root_gini):

    
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [55]:


def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [56]:

def find_best_split(data, labels):
    
    
    min_samples_leaf = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [57]:


def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    
    return Node(index, t, true_branch, false_branch)

In [58]:
def classify_object(obj, node):

    
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [59]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [60]:
tree=build_tree(X_train,y_train)

In [61]:
validation_predictions=predict(X_validation,tree)

In [62]:
def coefficient_of_determination(y,y_predicted):
    ss_tot=np.sum((y-y.mean())**2)
    ss_res=np.sum((y-y_predicted)**2)
    return 1-(ss_res/ss_tot)

In [63]:
#Результат без PCA
cod_val=coefficient_of_determination(y_validation,validation_predictions)
cod_val

0.9626352015732547

____________________________

In [64]:
#Ниже проверем результат с PCA

In [65]:
#Проведем масшибирование
def standartisation(dataset):
    result=(dataset-dataset.mean(axis=0))/dataset.std(axis=0)
    return result

In [66]:
iris_data_standart=standartisation(iris_data.copy())

In [67]:
cov_maxtrix=iris_data_standart.T@iris_data_standart

In [68]:
own_values,own_vectors=np.linalg.eig(cov_maxtrix)

In [69]:
li_=[(np.abs(own_values[n]),own_vectors[n]) for n in range(own_vectors.shape[1])]

In [70]:
var_list=[(i[0]/np.sum(own_values)) * 100 for i in li_]
var_list

[72.96244541329987, 22.850761786701774, 3.66892188928287, 0.5178709107154803]

In [71]:
#Сформируем вектор весов и собственных векторов, соответствующих двум первым компонентам

In [72]:
own_vectors[:1]

array([[ 0.52106591, -0.37741762, -0.71956635,  0.26128628]])

In [73]:
Z_2=np.dot(iris_data_standart,own_vectors[:,:2])

In [74]:
Z_2[:5]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538]])

In [34]:
X_train_pca,X_validation_pca,y_train_pca,y_validation_pca=train_test_split(Z_2,iris_target,test_size=0.25,random_state=42)

In [35]:
tree_pca=build_tree(X_train_pca,y_train_pca)

In [36]:
validation_predictions_pca=predict(X_validation_pca,tree_pca)

In [37]:
#Результат PCA от 2-х признаков демонстрирует снижение к-т детерминации на валидационной выборке по сравнению с предсказаниями без PCA
cod_val_pca=coefficient_of_determination(y_validation_pca,validation_predictions_pca)
cod_val_pca

0.887905604719764

In [38]:
#Проведем оценку для 3-х признаков

In [39]:
Z_3=np.dot(iris_data_standart,own_vectors[:,:3])

In [40]:
X_train_pca,X_validation_pca,y_train_pca,y_validation_pca=train_test_split(Z_3,iris_target,test_size=0.25,random_state=42)

In [41]:
tree_pca=build_tree(X_train_pca,y_train_pca)

In [42]:
validation_predictions_pca=predict(X_validation_pca,tree_pca)

In [43]:
#Результат PCA от 3-х признаков демонстрирует к-т детерминации  равный к-т детерминации на валидационной выборке без PCA,
#что показывает бесполезность 4-го признака для обучения
cod_val_pca=coefficient_of_determination(y_validation_pca,validation_predictions_pca)
cod_val_pca

0.9626352015732547